this script prepares the data for regression analysis

In [1]:
import pandas as pd
import numpy as np
import sys

In [51]:
income_df_path='../../data/dropbox/Data/Additional_data/cntry_year_gdp.csv'
flag_df_path='../../data/dropbox/Data/Additional_data/country_WoS_ECI_WB_flags.tsv'
gini_df_path = '../../data/dropbox/Data/Derived/Publication_based/Gini/Normalized/agg_gini_full.csv'
pub_cnt_path='../../data/dropbox/Data/Derived/Cleaned_raw_data/pubcnt_full.csv'
dscp_df_path="../../data/dropbox/Data/Derived/Publication_based/LEIDEN/leiden_full_1973-2017.csv"
dissim_df_path="../../exps/Transition/density_norm.csv"
eci_df_path="../../data/dropbox/Data/Additional_data/eci_country_rankings.csv"

In [69]:
meta_output_path="regression.csv"

In [ ]:
print("the path is: "+income_df_path)

In [ ]:
def reshape_data(data,namelist):
    #reshape the concerned indicator to [cntry,year,value in time 0,value in time 1]
    result_df=pd.DataFrame()
    for index, year in enumerate(yearlist[:-1]):
        df_t0=data[data.Year==year]
        name_t0=[x+"_t0" for x in namelist]
        df_t0=df_t0.rename(columns=dict(zip(namelist,name_t0)))
        
        df_t1=data[data.Year==yearlist[index+1]]
        name_t1=[x+"_t1" for x in namelist]
        df_t1=df_t1.rename(columns=dict(zip(namelist,name_t1)))
        
        df=df_t0.merge(df_t1.drop(labels='Year',axis=1),on='Code')
        result_df=pd.concat([result_df,df])
    
    return result_df

In [3]:
income_df=pd.read_csv(income_df_path)
income_df.head()

,Code,Year,Income
0,ABW,1973-1977,NaN
1,AFG,1973-1977,2.352889e+09
2,AGO,1973-1977,NaN
3,ALB,1973-1977,NaN
4,AND,1973-1977,2.077614e+08


In [6]:
yearlist=income_df.Year.unique()
yearlist.sort()
yearlist

array(['1973-1977', '1978-1982', '1983-1987', '1988-1992', '1993-1997',
       '1998-2002', '2003-2007', '2008-2012', '2013-2017'], dtype=object)

In [8]:
income_df_reshape=reshape_data(income_df,['Income'])
income_df_reshape.head()

,Code,Year,Income_t0,Income_t1
0,ABW,1973-1977,NaN,NaN
1,AFG,1973-1977,2.352889e+09,3.529613e+09
2,AGO,1973-1977,NaN,5.677156e+09
3,ALB,1973-1977,NaN,NaN
4,AND,1973-1977,2.077614e+08,3.861716e+08


In [12]:
flag_df=pd.read_csv(flag_df_path,sep="\t")
flag_df.head()

,WoS,ECI_Country,WB,Code
0,Albania,Albania,Albania,ALB
1,Algeria,Algeria,Algeria,DZA
2,Angola,Angola,Angola,AGO
3,Argentina,Argentina,Argentina,ARG
4,Australia,Australia,Australia,AUS


In [15]:
gini_df=pd.read_csv(gini_df_path)
gini_df=gini_df.merge(flag_df,left_on='COUNTRY',right_on='WoS')
gini_df=gini_df.rename(columns={'YEAR':'Year'})[['Code','Year','GINI']]
gini_reshape=reshape_data(gini_df,['GINI'])
gini_reshape.head()

,Code,Year,GINI_t0,GINI_t1
0,AFG,1973-1977,0.910166,0.936461
1,DZA,1973-1977,0.775474,0.670606
2,AGO,1973-1977,0.950697,0.977281
3,ATA,1973-1977,0.986057,0.983091
4,ARG,1973-1977,0.591460,0.562748


In [20]:
gini_reshape['diversity_t0']=1-gini_reshape['GINI_t0']
gini_reshape['diversity_t1']=1-gini_reshape['GINI_t1']
gini_reshape.head()

,Code,Year,GINI_t0,GINI_t1,diversity_t0,diversity_t1
0,AFG,1973-1977,0.910166,0.936461,0.089834,0.063539
1,DZA,1973-1977,0.775474,0.670606,0.224526,0.329394
2,AGO,1973-1977,0.950697,0.977281,0.049303,0.022719
3,ATA,1973-1977,0.986057,0.983091,0.013943,0.016909
4,ARG,1973-1977,0.591460,0.562748,0.408540,0.437252


In [22]:
pub_cnt=pd.read_csv(pub_cnt_path)
pub_cnt=pub_cnt.merge(flag_df[['WoS','Code']],left_on='COUNTRY',right_on='WoS')
pub_cnt=pub_cnt.rename(columns={'YEAR':'Year','PAPER_CNT':'pub_cnt'})
pub_cnt.head()

,Year,DISCIPLINE,SPECIALTY,COUNTRY,pub_cnt,WoS,Code
0,1973,Arts,Fine Arts & Architecture,Austria,1,Austria,AUT
1,1973,Biology,Agricult & Food Science,Austria,11,Austria,AUT
2,1973,Biology,Botany,Austria,46,Austria,AUT
3,1973,Biology,Ecology,Austria,6,Austria,AUT
4,1973,Biology,General Zoology,Austria,2,Austria,AUT


In [23]:
def get_pub_year(pubdata,groupbylist):
    cntry_period_df=pd.DataFrame()
    for year in yearlist:
        start,end=year.split("-")
        data=pubdata[pubdata['Year'].between(int(start),int(end))]
        data=data.groupby(groupbylist)['pub_cnt'].sum().reset_index()
        data['pub_cnt']=data['pub_cnt']/5
        data['Year']=year
        cntry_period_df=pd.concat([data,cntry_period_df])
    return cntry_period_df

In [24]:
cntry_cnt = get_pub_year(pub_cnt,['Code'])
cntry_cnt_reshape=reshape_data(cntry_cnt,['pub_cnt'])
cntry_cnt_reshape.head()

,Code,pub_cnt_t0,Year,pub_cnt_t1
0,AFG,8.8,1973-1977,7.8
1,AGO,4.8,1973-1977,1.0
2,ARE,1.2,1973-1977,9.6
3,ARG,873.8,1973-1977,1110.0
4,ATA,1.2,1973-1977,1.2


In [42]:
dscp_df=pd.read_csv(dscp_df_path,sep="\t",names=['SPECIALTY','group'])
dscp_cnt=pub_cnt.merge(dscp_df,on='SPECIALTY')
dscp_cnt=get_pub_year(dscp_cnt,['Code','group'])
dscp_cnt=dscp_cnt.pivot_table(index=['Code','Year'],columns='group',values='pub_cnt').reset_index()
dscp_cnt_reshape=reshape_data(dscp_cnt,['NE','NM','SHM'])
dscp_cnt_reshape.head()

group,Code,Year,NE_t0,NM_t0,SHM_t0,NE_t1,NM_t1,SHM_t1
0,AFG,1973-1977,1.8,5.4,1.6,1.0,6.6,0.2
1,AGO,1973-1977,2.0,2.0,0.8,0.4,0.6,NaN
2,ARE,1973-1977,0.4,0.4,0.4,3.8,3.4,2.4
3,ARG,1973-1977,255.0,308.6,310.2,374.2,451.0,284.8
4,ATA,1973-1977,NaN,1.2,NaN,NaN,0.8,0.4


In [70]:
dissim_df=pd.read_csv(dissim_df_path)
dissim_df=dissim_df.rename(columns={'COUNTRY':'WoS','CRRT_TIME':'Year','Density_norm':'Similarity'})
dissim_df=dissim_df.merge(flag_df[['Code','WoS']],on='WoS')
dissim_df.head()

,WoS,Year,Similarity,Code
0,Afghanistan,1973-1977,1.271279,AFG
1,Afghanistan,1978-1982,1.165540,AFG
2,Afghanistan,1983-1987,0.599289,AFG
3,Afghanistan,1988-1992,0.330956,AFG
4,Afghanistan,1993-1997,0.458757,AFG


In [50]:
def construct_eciperiod(data, yearlist):
    eciperiod=pd.DataFrame()
    for year in yearlist:
        start,end=year.split("-")
        data_filter=data[data.Year.between(int(start),int(end))]
        data_filter=data_filter.groupby('Code')['ECI'].mean().reset_index()
        data_filter['Year']=year
        eciperiod=pd.concat([eciperiod, data_filter])
    return eciperiod


In [62]:
eci_df=pd.read_csv(eci_df_path)
eci_df=eci_df.merge(flag_df[['ECI_Country','Code']],
          left_on='Country',right_on='ECI_Country',how='inner')
eci_df = construct_eciperiod(eci_df,yearlist)
eci_df.head()

,Code,ECI,Year
0,AGO,-1.291844,1973-1977
1,ALB,-0.162595,1973-1977
2,ARE,-0.200329,1973-1977
3,ARG,0.218150,1973-1977
4,AUS,-0.032383,1973-1977


In [63]:
eci_reshape=reshape_data(eci_df,['ECI'])
eci_reshape.head()

,Code,ECI_t0,Year,ECI_t1
0,AGO,-1.291844,1973-1977,-1.566572
1,ALB,-0.162595,1973-1977,0.192658
2,ARE,-0.200329,1973-1977,-0.053862
3,ARG,0.218150,1973-1977,-0.031744
4,AUS,-0.032383,1973-1977,-0.066588


In [71]:
metadata=cntry_cnt_reshape.merge(dscp_cnt_reshape,on=['Code','Year'],how="left")
metadata=metadata.merge(gini_reshape[['Code','Year','diversity_t0','diversity_t1']],
                       on=['Code','Year'],how='left')
metadata=metadata.merge(income_df_reshape,on=['Code','Year'],how='left')
metadata=metadata.merge(eci_reshape,on=['Code','Year'],how='left')
metadata=metadata.merge(dissim_df,on=['Code','Year'],how='left')
metadata.head()

,Code,pub_cnt_t0,Year,pub_cnt_t1,NE_t0,NM_t0,SHM_t0,NE_t1,NM_t1,SHM_t1,diversity_t0,diversity_t1,Income_t0,Income_t1,ECI_t0,ECI_t1,WoS,Similarity
0,AFG,8.8,1973-1977,7.8,1.8,5.4,1.6,1.0,6.6,0.2,0.089834,0.063539,2.352889e+09,3.529613e+09,NaN,NaN,Afghanistan,1.271279
1,AGO,4.8,1973-1977,1.0,2.0,2.0,0.8,0.4,0.6,NaN,0.049303,0.022719,NaN,5.677156e+09,-1.291844,-1.566572,Angola,0.665151
2,ARE,1.2,1973-1977,9.6,0.4,0.4,0.4,3.8,3.4,2.4,0.024098,0.107890,1.960182e+10,3.891124e+10,-0.200329,-0.053862,United Arab Emirates,0.367671
3,ARG,873.8,1973-1977,1110.0,255.0,308.6,310.2,374.2,451.0,284.8,0.408540,0.437252,5.707399e+10,7.345629e+10,0.218150,-0.031744,Argentina,0.413694
4,ATA,1.2,1973-1977,1.2,NaN,1.2,NaN,NaN,0.8,0.4,0.013943,0.016909,NaN,NaN,NaN,NaN,Antarctica,0.518377


In [72]:
metadata['pub_growth']=np.log10(metadata['pub_cnt_t1']/metadata['pub_cnt_t0'])
metadata['income_growth']=np.log10(metadata['Income_t1']/metadata['Income_t0'])
metadata['diversity_growth']=metadata['diversity_t1']/metadata['diversity_t0']

In [73]:
metadata['pub_cnt_t0']=np.log10(metadata['pub_cnt_t0'])
metadata['pub_cnt_t1']=np.log10(metadata['pub_cnt_t1'])
metadata['Income_t0']=np.log10(metadata['Income_t0'])
metadata['Income_t1']=np.log10(metadata['Income_t1'])
metadata['NE_t0']=np.log10(metadata['NE_t0'])
metadata['NM_t0']=np.log10(metadata['NM_t0'])
metadata['SHM_t0']=np.log10(metadata['SHM_t0'])
metadata['NE_t1']=np.log10(metadata['NE_t1'])
metadata['NM_t1']=np.log10(metadata['NM_t1'])
metadata['SHM_t1']=np.log10(metadata['SHM_t1'])
metadata['date']=metadata['Year'].apply(lambda x:list(yearlist).index(x))
metadata.head()

,Code,pub_cnt_t0,Year,pub_cnt_t1,NE_t0,NM_t0,SHM_t0,NE_t1,NM_t1,SHM_t1,...,Income_t0,Income_t1,ECI_t0,ECI_t1,WoS,Similarity,pub_growth,income_growth,diversity_growth,date
0,AFG,0.944483,1973-1977,0.892095,0.255273,0.732394,0.204120,0.000000,0.819544,-0.698970,...,9.371601,9.547727,NaN,NaN,Afghanistan,1.271279,-0.052388,0.176126,0.707295,0
1,AGO,0.681241,1973-1977,0.000000,0.301030,0.301030,-0.096910,-0.397940,-0.221849,NaN,...,NaN,9.754131,-1.291844,-1.566572,Angola,0.665151,-0.681241,NaN,0.460797,0
2,ARE,0.079181,1973-1977,0.982271,-0.397940,-0.397940,-0.397940,0.579784,0.531479,0.380211,...,10.292296,10.590075,-0.200329,-0.053862,United Arab Emirates,0.367671,0.903090,0.297779,4.477207,0
3,ARG,2.941412,1973-1977,3.045323,2.406540,2.489396,2.491642,2.573104,2.654177,2.454540,...,10.756438,10.866029,0.218150,-0.031744,Argentina,0.413694,0.103911,0.109591,1.070279,0
4,ATA,0.079181,1973-1977,0.079181,NaN,0.079181,NaN,NaN,-0.096910,-0.397940,...,NaN,NaN,NaN,NaN,Antarctica,0.518377,0.000000,NaN,1.212721,0


In [ ]:
metadata.to_csv(meta_output_path,index=False)